# 🚀 SOLUCIÓN FINAL: 100% RDD Sin Timeout
## Optimizado, Robusto y Probado en Windows

In [1]:
import os
import warnings
warnings.filterwarnings('ignore')

os.environ["HADOOP_HOME"] = "C:\\hadoop"
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

import numpy as np
import time
from datetime import datetime

print("✓ Librerías importadas")

✓ Librerías importadas


In [2]:
#----------------SparkSession CON TIMEOUTS AUMENTADOS-------------------------
spark = SparkSession.builder \
    .appName("DeepLearning_100RDD_FINAL") \
    .master("local[8]") \
    .config("spark.driver.memory", "12g") \
    .config("spark.executor.memory", "12g") \
    .config("spark.driver.maxResultSize", "8g") \
    .config("spark.network.timeout", "800s") \
    .config("spark.executor.heartbeatInterval", "60s") \
    .config("spark.python.worker.timeout", "600s") \
    .config("spark.sql.shuffle.partitions", "16") \
    .config("spark.default.parallelism", "16") \
    .config("spark.rdd.compress", "true") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

print("✓ Spark con timeouts aumentados")
print("  Network timeout: 800s")
print("  Worker timeout: 600s")

✓ Spark con timeouts aumentados
  Network timeout: 800s
  Worker timeout: 600s


In [3]:
#----------------------Cargar datos-----------------------------------------------
DATA_PATH = "C:/Users/PC/Documents/DocumentosGustavo/Github/Maestria/BigData/nyc-taxi-spark/data/yellow/2024/yellow_tripdata_2024-01.parquet"

print("\nCargando dataset...")
df = spark.read.parquet(DATA_PATH)
print(f"✓ Dataset: {df.count():,} registros")


Cargando dataset...
✓ Dataset: 2,964,624 registros


In [4]:
#----------------------Feature Engineering----------------------------------------
def extract_and_scale_features(row):
    trip_distance, passenger_count, datetime, fare_amount = row
    
    if (trip_distance is None or trip_distance <= 0 or trip_distance >= 100 or
        passenger_count is None or passenger_count <= 0 or passenger_count > 6 or
        datetime is None or
        fare_amount is None or fare_amount <= 0 or fare_amount >= 200):
        return None
    
    hour_value = float(datetime.hour)
    day_of_week = float(datetime.weekday() + 1)
    
    features = [
        float((trip_distance - 3.0) / 5.0),
        float((passenger_count - 1.5) / 1.0),
        float((hour_value - 12.0) / 7.0),
        float((day_of_week - 4.0) / 2.0)
    ]
    
    return (features, float(fare_amount))

print("\nProcesando features...")
start = time.time()

rdd_features = df.select(
    "trip_distance", "passenger_count", "tpep_pickup_datetime", "fare_amount"
).rdd.map(lambda row: (
    row.trip_distance, row.passenger_count, row.tpep_pickup_datetime, row.fare_amount
))

rdd_scaled = rdd_features \
    .map(extract_and_scale_features) \
    .filter(lambda x: x is not None) \
    .repartition(16) \
    .cache()

total_scaled = rdd_scaled.count()
print(f"✓ Completado en {time.time()-start:.1f}s: {total_scaled:,} registros")


Procesando features...
✓ Completado en 87.4s: 2,722,784 registros


In [5]:
#----------------------División Train/Test----------------------------------------
train_rdd, test_rdd = rdd_scaled.randomSplit([0.8, 0.2], seed=42)

from pyspark import StorageLevel
train_rdd = train_rdd.repartition(16).persist(StorageLevel.MEMORY_AND_DISK)
test_rdd = test_rdd.repartition(8).persist(StorageLevel.MEMORY_AND_DISK)

train_count = train_rdd.count()
test_count = test_rdd.count()

print(f"\n✓ Train: {train_count:,}")
print(f"✓ Test: {test_count:,}")


✓ Train: 2,177,985
✓ Test: 544,799


In [6]:
#----------------------Modelo-----------------------------------------------------
def create_model():
    model = Sequential([
        Dense(64, activation='relu', input_shape=(4,)),
        BatchNormalization(),
        Dropout(0.2),
        Dense(32, activation='relu'),
        BatchNormalization(),
        Dropout(0.2),
        Dense(16, activation='relu'),
        Dense(8, activation='relu'),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer=Adam(0.001), loss='mse', metrics=['mae'])
    return model

model = create_model()
print("\n✓ Modelo creado")
model.summary()


✓ Modelo creado


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,457 (13.50 KB)

 Trainable params: 3,265 (12.75 KB)

 Non-trainable params: 192 (768.00 B)

In [7]:
#----------------------Generador ROBUSTO (Sin Timeout)----------------------------
print("\n" + "="*80)
print("GENERADOR DE BATCHES ROBUSTO - SIN TIMEOUT")
print("="*80)

class RobustRDDBatchGenerator:
    """
    Generador que usa toLocalIterator() en lugar de collect().
    
    VENTAJAS:
    - No hay timeout (itera sin collect masivo)
    - Más eficiente en memoria
    - Más robusto para datasets grandes
    - Sigue siendo 100% RDD
    """
    
    def __init__(self, rdd, batch_size=4096, num_batches_per_epoch=None):
        self.rdd = rdd
        self.batch_size = batch_size
        self.total_samples = rdd.count()
        
        if num_batches_per_epoch:
            self.num_batches = num_batches_per_epoch
        else:
            self.num_batches = max(1, self.total_samples // batch_size)
    
    def generate_batches(self, seed=42):
        """
        Genera batches usando toLocalIterator.
        
        toLocalIterator:
        - Itera sobre RDD SIN collect masivo
        - No causa timeout
        - Procesa partición por partición
        - 100% RDD distribuido
        """
        # Sample del RDD
        fraction = min(1.0, (self.batch_size * self.num_batches) / self.total_samples)
        sampled_rdd = self.rdd.sample(False, fraction, seed=seed)
        
        # Usar toLocalIterator en lugar de collect
        batch_data = []
        batch_count = 0
        
        # Iterar sin collect masivo
        for item in sampled_rdd.toLocalIterator():
            batch_data.append(item)
            
            # Cuando el batch está lleno, yield
            if len(batch_data) >= self.batch_size:
                X_batch = np.array([x[0] for x in batch_data], dtype=np.float32)
                y_batch = np.array([x[1] for x in batch_data], dtype=np.float32)
                
                yield X_batch, y_batch
                
                batch_data = []
                batch_count += 1
                
                # Limitar número de batches
                if batch_count >= self.num_batches:
                    break
        
        # Último batch parcial
        if batch_data and batch_count < self.num_batches:
            X_batch = np.array([x[0] for x in batch_data], dtype=np.float32)
            y_batch = np.array([x[1] for x in batch_data], dtype=np.float32)
            yield X_batch, y_batch

# Configuración
BATCH_SIZE = 4096  # Un poco más pequeño para ser más robusto
BATCHES_PER_EPOCH_TRAIN = 400
BATCHES_PER_EPOCH_VAL = 20

train_generator = RobustRDDBatchGenerator(
    train_rdd, 
    batch_size=BATCH_SIZE,
    num_batches_per_epoch=BATCHES_PER_EPOCH_TRAIN
)

test_generator = RobustRDDBatchGenerator(
    test_rdd,
    batch_size=BATCH_SIZE,
    num_batches_per_epoch=BATCHES_PER_EPOCH_VAL
)

print("\n✓ Generador robusto configurado")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Batches/época train: {BATCHES_PER_EPOCH_TRAIN}")
print(f"  Samples/época: {BATCH_SIZE * BATCHES_PER_EPOCH_TRAIN:,}")
print(f"\n💡 Usa toLocalIterator() - Sin timeout, 100% RDD")


GENERADOR DE BATCHES ROBUSTO - SIN TIMEOUT

✓ Generador robusto configurado
  Batch size: 4096
  Batches/época train: 400
  Samples/época: 1,638,400

💡 Usa toLocalIterator() - Sin timeout, 100% RDD


In [8]:
#----------------------Entrenamiento----------------------------------------------
print("\n" + "="*80)
print("ENTRENAMIENTO 100% RDD - ROBUSTO")
print("="*80)

EPOCHS = 15

print(f"\n⚙️  Configuración:")
print(f"   Épocas: {EPOCHS}")
print(f"   Batches/época: {BATCHES_PER_EPOCH_TRAIN}")
print(f"   Batch size: {BATCH_SIZE}")

history = {'loss': [], 'mae': [], 'val_loss': [], 'val_mae': []}

print("\n🎯 Iniciando...\n")
start_time = time.time()

for epoch in range(EPOCHS):
    epoch_start = time.time()
    print(f"\nÉpoca {epoch+1}/{EPOCHS}")
    print("-" * 60)
    
    epoch_losses = []
    epoch_maes = []
    
    # Entrenar
    batch_count = 0
    try:
        for X_batch, y_batch in train_generator.generate_batches(seed=epoch):
            metrics = model.train_on_batch(X_batch, y_batch, return_dict=True)
            epoch_losses.append(metrics['loss'])
            epoch_maes.append(metrics['mae'])
            batch_count += 1
            
            if batch_count % 100 == 0:
                print(f"  Batch {batch_count}/{BATCHES_PER_EPOCH_TRAIN} - "
                      f"loss: {np.mean(epoch_losses[-20:]):.4f}")
    except Exception as e:
        print(f"⚠️  Error en batch {batch_count}: {e}")
        print(f"  Continuando con siguiente época...")
        continue
    
    if not epoch_losses:
        print("  No se completaron batches, saltando época")
        continue
    
    train_loss = np.mean(epoch_losses)
    train_mae = np.mean(epoch_maes)
    
    # Validación
    val_losses = []
    val_maes = []
    for X_val, y_val in test_generator.generate_batches(seed=epoch):
        val_metrics = model.test_on_batch(X_val, y_val, return_dict=True)
        val_losses.append(val_metrics['loss'])
        val_maes.append(val_metrics['mae'])
    
    val_loss = np.mean(val_losses) if val_losses else train_loss
    val_mae = np.mean(val_maes) if val_maes else train_mae
    
    history['loss'].append(train_loss)
    history['mae'].append(train_mae)
    history['val_loss'].append(val_loss)
    history['val_mae'].append(val_mae)
    
    epoch_time = time.time() - epoch_start
    print(f"\n  📊 Época {epoch+1}:")
    print(f"     loss: {train_loss:.4f} - mae: {train_mae:.4f}")
    print(f"     val_loss: {val_loss:.4f} - val_mae: {val_mae:.4f}")
    print(f"     Tiempo: {epoch_time:.1f}s")
    
    # Early stopping
    if epoch > 3 and val_loss > history['val_loss'][-2]:
        patience = getattr(model, 'patience', 0) + 1
        model.patience = patience
        if patience >= 3:
            print(f"\n⚠️  Early stopping")
            break
    else:
        model.patience = 0

training_time = time.time() - start_time

print("\n" + "="*80)
print("✓ ENTRENAMIENTO COMPLETADO")
print("="*80)
print(f"  Tiempo: {training_time/60:.2f} minutos")
print(f"  Épocas: {len(history['loss'])}")
print(f"  Mejor val_loss: {min(history['val_loss']):.4f}")


ENTRENAMIENTO 100% RDD - ROBUSTO

⚙️  Configuración:
   Épocas: 15
   Batches/época: 400
   Batch size: 4096

🎯 Iniciando...


Época 1/15
------------------------------------------------------------
  Batch 100/400 - loss: 469.1248
  Batch 200/400 - loss: 264.1448
  Batch 300/400 - loss: 186.9661
  Batch 400/400 - loss: 148.8105

  📊 Época 1:
     loss: 315.1734 - mae: 11.3691
     val_loss: 143.1372 - val_mae: 6.7481
     Tiempo: 62.1s

Época 2/15
------------------------------------------------------------
  Batch 100/400 - loss: 122.0138
  Batch 200/400 - loss: 107.8552
  Batch 300/400 - loss: 97.4814
  Batch 400/400 - loss: 89.6862

  📊 Época 2:
     loss: 109.2100 - mae: 5.6545
     val_loss: 88.2205 - val_mae: 4.9640
     Tiempo: 57.4s

Época 3/15
------------------------------------------------------------
  Batch 100/400 - loss: 82.2312
  Batch 200/400 - loss: 77.4876
  Batch 300/400 - loss: 73.4865
  Batch 400/400 - loss: 70.1736

  📊 Época 3:
     loss: 77.5771 - mae: 4.6030

In [9]:
#----------------------Evaluación-------------------------------------------------
print("\nEvaluando modelo...")

eval_generator = RobustRDDBatchGenerator(
    test_rdd,
    batch_size=4096,
    num_batches_per_epoch=100
)

all_predictions = []
all_actuals = []

for X_test, y_test_batch in eval_generator.generate_batches(seed=99):
    y_pred = model.predict(X_test, verbose=0)
    all_predictions.extend(y_pred.flatten().tolist())
    all_actuals.extend(y_test_batch.tolist())

y_test_eval = np.array(all_actuals)
y_pred_eval = np.array(all_predictions)

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

mse = mean_squared_error(y_test_eval, y_pred_eval)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test_eval, y_pred_eval)
r2 = r2_score(y_test_eval, y_pred_eval)
mape = np.mean(np.abs((y_test_eval - y_pred_eval) / y_test_eval)) * 100

print("\n" + "="*80)
print("RESULTADOS")
print("="*80)
print(f"\n  R²:   {r2:.4f} ({r2*100:.1f}%)")
print(f"  RMSE: ${rmse:.4f}")
print(f"  MAE:  ${mae:.4f}")
print(f"  MAPE: {mape:.2f}%")
print(f"\n  Evaluado en {len(y_test_eval):,} predicciones")


Evaluando modelo...

RESULTADOS

  R²:   0.8876 (88.8%)
  RMSE: $5.5770
  MAE:  $2.6594
  MAPE: 20.59%

  Evaluado en 409,208 predicciones


In [ ]:
#----------------------Guardar----------------------------------------------------
#os.makedirs("modelos", exist_ok=True)
#model_path = f"modelos/taxi_100RDD_ROBUSTO_{datetime.now().strftime('%Y%m%d_%H%M%S')}.h5"
#model.save(model_path)
#print(f"\n✓ Modelo guardado: {model_path}")